In [ ]:
import mdtraj as md 

#load trajectories variant (RS) and wildtype (WT)
t_RS = md.load('RS_variant/run.xtc', top='RS_variant/protein.gro')
t_wt = md.load('wildtype/run.xtc', top='wildtype/protein.gro')

top_RS=t_RS.topology
top_wt = t_wt.topology

# atom-selection of loop-CA atoms
loop_RS = top_RS.select("(resid 389 to 405) and name CA")
loop_wt = top_wt.select("(resid 389 to 405) and name CA")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#RMSF analysis

#superimpose
t_RSsuper = t_RS.superpose(t_RS,frame=0, atom_indices=protein_RS)
t_wtsuper = t_wt.superpose(t_wt,frame=0, atom_indices=protein_wt)
#calculate rmsf
rmsf_RS = md.rmsf(t_RSsuper, None, atom_indices=loop_RS)
rmsf_wt = md.rmsf(t_wtsuper, None, atom_indices=loop_wt)

#plot rmsf
x = [i for i in range(389,406)]
df = pd.DataFrame({'positions':x, 'rmsf_wt': rmsf_wt, 'rmsf_RS':rmsf_RS})
cm = 1/2.54
rcParams['svg.fonttype'] = 'none'

fig, ax1 = plt.subplots(figsize=(8*cm, 6.5*cm))

tidy = df.melt(id_vars='positions').rename(columns=str.title)
sns.set_style('white', {'legend.frameon':False, 'axes.grid': False})
sns.barplot(x='Positions', y='Value', hue='Variable', data=tidy, ax=ax1, palette=['black', 'steelblue'])
sns.despine(fig)
ax1.set_xlabel('position',size=10)
ax1.set_ylabel('RMSF in Ang',size=10)
plt.tick_params(left = False, right = False , labelleft = True , 
                labelbottom = True, bottom = True) 
plt.xticks(rotation=90, size=8)
plt.yticks( size=8)

sns.set_context("paper")
fig.tight_layout()
fig.savefig('RMSF_sims.png', dpi=300)
fig.savefig('RMSF_sims.svg', format='svg')

In [1]:
import numpy as np
from sklearn.decomposition import PCA
from itertools import combinations


pca = PCA(n_components=2)

#compute pairwise CA-distances
atom_pairs_wt = list(combinations(range(min(loop_wt),max(loop_wt)+1), 2))
pairwise_distances_wt = md.geometry.compute_distances(t_wt, atom_pairs_wt)
print(pairwise_distances_wt.shape)


atom_pairs_RS = list(combinations(range(min(loop_RS),max(loop_RS)+1), 2))
pairwise_distances_RS = md.geometry.compute_distances(t_RS, atom_pairs_RS)
print(pairwise_distances_RS.shape)

#combine distance matrices from wt and variant and perform PCA
combined_distances = np.concatenate((pairwise_distances_wt, pairwise_distances_RS), axis=0)
reduced_distances = pca.fit_transform(combined_distances)


In [ ]:
#plot PCA analysis as kde plots

import matplotlib.patches as mpatches
from matplotlib import rcParams

cm = 1/2.54
rcParams['svg.fonttype'] = 'none'
fig,ax = plt.subplots(figsize=(6.5*cm, 6.5*cm))

sns.kdeplot(x=reduced_distances[:20001, 0], y=reduced_distances[:20001,1], color='black', linewidths=0.5,  fill=True, alpha=0.6)
sns.kdeplot(x=reduced_distances[20002:, 0], y=reduced_distances[20002:,1],  color='steelblue', linewidths=0.5, fill=True, alpha=0.6)
plt.xlabel('PC1', size=10)
plt.ylabel('PC2', size=10)

plt.tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False) 
red_patch = mpatches.Patch(color='steelblue', label='R425S variant')
blue_patch = mpatches.Patch(color='black', label='WT')
plt.legend(handles=[blue_patch,red_patch],fontsize=7, loc='upper right', ncols=1, markerscale=0.5,  frameon=False)

sns.despine()
sns.set_context("paper")
fig.tight_layout()

fig.savefig('PCA_wt_vs_R425S.png', dpi=300)
fig.savefig('PCA_wt_vs_R425S.svg')

In [ ]:
#perform DBSCAN clustering on PCA output
from sklearn.cluster import DBSCAN
dbscan_fit = DBSCAN(eps=2, min_samples=280, n_jobs=20).fit(reduced_distances)
dbscan = dbscan_fit.labels_

In [ ]:
from palettable.cartocolors.qualitative import Bold_6
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D

#import palette for cluster coloring
palette = sns.color_palette(Bold_6.mpl_colors)

clusterlabels = dbscan
method = 'dbscan'

cm = 1/2.54
fig,ax = plt.subplots(figsize=(6.5*cm, 6.5*cm))

#add kde plots of PCA in background
sns.kdeplot(x=reduced_distances[:20001, 0], y=reduced_distances[:20001,1], color='black', fill=True, alpha=0.5)
sns.kdeplot(x=reduced_distances[20001:, 0], y=reduced_distances[20001:,1],  color='steelblue', fill=True, alpha=0.5)

#display not clustered points ("outliers") as x and clusters as one-line kde plot 
clusterlist = list( dict.fromkeys(clusters) )
for num, cluster in enumerate(clusterlist):
    if num==1: 
        indices = np.where(clusterlabels == cluster)[0]
        points_cluster = np.array([reduced_distances[i] for i in indices])
        sns.scatterplot(x=points_cluster[:,0], y=points_cluster[:,1], marker='x', alpha=1, s=2, 
                color='slategrey',  legend=False)
    else:
        indices = np.where(clusterlabels == cluster)[0]
        points_cluster = np.array([reduced_distances[i] for i in indices])
        sns.kdeplot(x=points_cluster[:, 0], y=points_cluster[:,1], color=palette[num], linewidths=1, levels=1, thresh=0.1)
        handles, labels = ax.get_legend_handles_labels()


plt.xlabel('PC1', size=10)
plt.ylabel('PC2', size=10)
plt.tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False) 

#create legend
cluster1 = Line2D([0], [0], label='Cluster 1', linewidth=1, color=palette[0])
cluster2 = Line2D([0], [0], label='Cluster 2', linewidth=1, color=palette[2])
cluster3 = Line2D([0], [0], label='Cluster 3', linewidth=1, color=palette[3])
cluster4 = Line2D([0], [0], label='Cluster 4', linewidth=1, color=palette[4])
cluster5 = Line2D([0], [0], label='Cluster 5', linewidth=1, color=palette[5])
outliers = Line2D([0], [0], label='Outliers', marker='x', markersize=6, color='slategrey', linestyle='')
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=[cluster1,cluster2,cluster3,cluster4,cluster5, outliers], 
           fontsize=7, loc='upper right', ncols=2, markerscale=0.5, frameon=False, columnspacing=1)

sns.despine()
sns.set_context("paper")
fig.tight_layout()
fig.savefig(f"{method}_clustering.png", dpi=300)
fig.savefig(f"{method}_clustering.svg", dpi=300)